导入库

In [1]:
import os
import imp
import numpy as np

import utils
imp.reload(utils)

/tmp/ipykernel_303426/1016847511.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'utils' from '/data1/byzhao/EEG-AudioTransformer/utils.py'>

公共变量

In [2]:
words_path = r'./feat/words'
pt = 'sub-06'
test_word = 10

读取数据

In [3]:
folder_path = os.path.join(words_path,f'{pt}')
filename = os.listdir(folder_path)[test_word]
word_info = np.load(os.path.join(folder_path,filename),allow_pickle=True)
word=word_info.item()['label']
eeg=word_info.item()['eeg']
audio=word_info.item()['audio']


数据预处理

In [4]:
window_length = 0.025
frameshift = 0.005
eeg_sample_rate = 1024
audio_sameple_rate = 16000

提取高频eeg信号和音频信号的梅尔频谱

In [5]:
eeg = utils.extractHG(eeg,eeg_sample_rate,windowLength=window_length,frameshift=frameshift)
audio = utils.extractMelSpecs(audio,audio_sameple_rate,windowLength=window_length,frameshift=frameshift)
if audio.shape[0]!=eeg.shape[0]:
    minlen = min(audio.shape[0],eeg.shape[0])
    audio = audio[:minlen,:]
    eeg = eeg[:minlen,:]

z均值处理

In [6]:
eeg_mean = np.mean(eeg)
eeg_std = np.std(eeg)
eeg = (eeg-eeg_mean)/eeg_std

加载模型

In [7]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

from torch.utils.data import TensorDataset, DataLoader

import transformer
imp.reload(transformer)
import json

/data1/byzhao/miniconda3/envs/bci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
config_path = r'./config'
model_name = '4h6l'

In [10]:
with open(os.path.join(config_path,f'{model_name}.json'),'r') as f:
    cfg = json.load(f)['model_config']

batch_size = cfg['batch_size']
epochs = cfg['epochs']
lr = cfg['lr']
b1 = cfg['b1']
b2 = cfg['b2']
scaled_dim = cfg['scaled_dim']
d_model = cfg['d_model']
nhead = cfg['nhead']
n_layer = cfg['n_layer']
input_dim = eeg.shape[1]
output_dim = audio.shape[1]

tensor_type = torch.cuda.FloatTensor

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# log_write = open(f"./log/log_{pt}.txt", "w") 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = transformer.Model(
    input_dim=input_dim,
    output_dim=output_dim,
    scaled_dim=scaled_dim,
    d_model=d_model,
    nhead=nhead,
    n_layer=n_layer
).to(device)

# criterion = nn.MSELoss(reduction='mean').to(device)
criterion = nn.L1Loss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=(b1,b2))

print(input_dim,output_dim)

127 23


In [12]:
# pbar = tqdm.trange(epochs, desc=f"Epochs")
model.load_state_dict(torch.load(f'./res/{pt}/{model_name}.pt')['model_state_dict'])
model.eval()

Model(
  (l1): Sequential(
    (0): Linear(in_features=127, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
  )
  (transformer): TransformerModel(
    (encoder_layer): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (linear1): Linear(in_features=256, out_features=1024, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=1024, out_features=256, bias=True)
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): Multihe

转换为MFCC

In [16]:
model_output = model(torch.from_numpy(eeg).to(device).type(tensor_type)).detach().cpu().numpy()

In [36]:
import matplotlib.pyplot as plt
import librosa
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f'./logs/{pt}/{model_name}')
origin_melspec_fig = plt.figure()
librosa.display.specshow(audio,sr=16000,x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')        
plt.title(f'{pt}-{word}-origin')
writer.add_figure(tag=f"{pt}-{word}-origin log Mel spectrogram",figure=origin_melspec_fig)

model_melspec_fig = plt.figure()
librosa.display.specshow(model_output,sr=16000,x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')        
plt.title(f'{pt}-{word}-model')
writer.add_figure(tag=f"{pt}-{word}-model log Mel spectrogram",figure=model_melspec_fig)
writer.close()

In [42]:
model_mfcc = utils.toMFCC(model_output)
mfcc = utils.toMFCC(audio)
eu_dis = 0
for i in range(mfcc.shape[0]):
    eu_dis += np.linalg.norm(model_mfcc[i] - mfcc[i])
mcd = eu_dis/mfcc.shape[0]
print(mcd)

1.1782167302323834
